## 新能源汽车数据爬取

本案例以新浪网新能源汽车分析数据为例，从新浪 http://db.auto.sina.com.cn 爬取车型列表、车型信息，为之后的可视化和情感分析建模做准备。

In [1]:
import csv, json, re, requests, json, time
import pandas as pd
import re
from bs4 import BeautifulSoup

In [2]:
url = "http://db.auto.sina.com.cn/estation/"
response = requests.get(url)
response.encoding = 'utf-8' 
soup = BeautifulSoup(response.text, 'html.parser')
brand_urls = [i['href'] for i in soup.select('.carlist .clearfix a')]
brand_urls[:6]

['//db.auto.sina.com.cn/eb1.html',
 '//db.auto.sina.com.cn/eb4.html',
 '//db.auto.sina.com.cn/eb3.html',
 '//db.auto.sina.com.cn/eb8.html',
 '//db.auto.sina.com.cn/eb9.html',
 '//db.auto.sina.com.cn/eb13.html']

In [3]:
brand_urls1 = []
for i in brand_urls:
    url = 'http:'+ str(i)
    brand_urls1.append(url)
brand_urls1[:6]

['http://db.auto.sina.com.cn/eb1.html',
 'http://db.auto.sina.com.cn/eb4.html',
 'http://db.auto.sina.com.cn/eb3.html',
 'http://db.auto.sina.com.cn/eb8.html',
 'http://db.auto.sina.com.cn/eb9.html',
 'http://db.auto.sina.com.cn/eb13.html']

In [4]:
brand_url = "http://db.auto.sina.com.cn/eb1.html"
response = requests.get(brand_url)
response.encoding = 'utf-8' 
soup = BeautifulSoup(response.text, 'html.parser')
model_urls = [i['href'] for i in soup.select('.ul .li')]
model_urls[:6]

['//db.auto.sina.com.cn/3460/',
 '//db.auto.sina.com.cn/3002/',
 '//db.auto.sina.com.cn/3854/',
 '//db.auto.sina.com.cn/4117/',
 '//db.auto.sina.com.cn/3694/',
 '//db.auto.sina.com.cn/2702/']

In [5]:
model_urls1 = []
for i in model_urls:
    url = 'http:'+ str(i)
    model_urls1.append(url)
model_urls1[:6]

['http://db.auto.sina.com.cn/3460/',
 'http://db.auto.sina.com.cn/3002/',
 'http://db.auto.sina.com.cn/3854/',
 'http://db.auto.sina.com.cn/4117/',
 'http://db.auto.sina.com.cn/3694/',
 'http://db.auto.sina.com.cn/2702/']

In [6]:
def crawl_model_urls_by_brand_url(brand_url):
    response = requests.get(brand_url)
    response.encoding = 'utf-8' 
    soup = BeautifulSoup(response.text, 'html.parser')
    model_urls = [i['href'] for i in soup.select('.ul .li')]
    model_urls[:6]
    model_urls1 = []
    for i in model_urls:
        url = 'http:'+ str(i)
        model_urls1.append(url)
    model_urls1[:6]

    return pd.DataFrame({'model_urls' : model_urls1})

In [7]:
brand_url = "http://db.auto.sina.com.cn/eb1.html"
model_urls1 = crawl_model_urls_by_brand_url(brand_url)
model_urls1

,model_urls
0,http://db.auto.sina.com.cn/3460/
1,http://db.auto.sina.com.cn/3002/
2,http://db.auto.sina.com.cn/3854/
3,http://db.auto.sina.com.cn/4117/
4,http://db.auto.sina.com.cn/3694/
5,http://db.auto.sina.com.cn/2702/
6,http://db.auto.sina.com.cn/3456/
7,http://db.auto.sina.com.cn/2995/
8,http://db.auto.sina.com.cn/2999/
9,http://db.auto.sina.com.cn/3000/


In [8]:
for brand_url in brand_urls1:
    model_urls = crawl_model_urls_by_brand_url(brand_url)
    model_urls.to_csv('D:/data/e-auto/model_urls.csv', mode = 'a', header = False, index = False)

启动网页CSS分析插件，选取相应的区域，得到不同信息的CSS路径为：

* 纯电续航（endurance_km）：.xuhang .t1；
* 慢充时间（slow_charge_hour）：.manchong .t1；
* 快充时间（fast_charge_hour）：.kuaichong .t1；
* 电池容量（capacity_kwh）：.dianchi .t1；
* 最大功率（max_power）：.gonglv .t1；
* 最大扭矩（max_torque）：.niuju .t1；
* 月销量（monthly_sales）：.monthsales .num；
* 年累计销量（yearly_sales）：.yearsales .num；
* 月成交价（monthly_price）：.monthprice .num；
* 用户评分（rating）：.star em；
* 口碑数量（review_count）：.txtnum；
* 厂商指导价格(guide_price) ： .strong。

定义函数`crawl_model_profiles_by_model_url()`，通过传入的汽车页面URL（`model_url`），返回数据框，包含纯电续航（`endurance_km`）、慢充时间（`slow_charge_hour`）、快充时间（`fast_charge_hour`）、电池容量（`capacity_kwh`）、最大功率（`max_power`）、最大扭矩（`max_torque`）、月销量（`monthly_sales`）、年累计销量（`yearly_sales`）、月成交价（`monthly_price`）、用户评分（`rating`）、口碑数量（`review_count`）和厂商指导价格(`guide_price`)。

In [9]:
def crawl_model_profiles_by_model_url(model_url):
    response = requests.get(model_url)
    response.encoding = 'utf-8' 
    soup = BeautifulSoup(response.text, 'html.parser')
    brand= soup.select('.brandBtn a')[0].text
    model=soup.select('.subBrandBtn a')[0].text
    endurance_km = soup.select('.xuhang .t1')[0].text if soup.select('.xuhang .t1')[0].text != '暂无数据' else ''
    slow_charge_hour = soup.select('.manchong .t1')[0].text if soup.select('.manchong .t1')[0].text != '暂无数据' else ''
    fast_charge_hour =soup.select('.kuaichong .t1')[0].text if soup.select('.kuaichong .t1')[0].text != '暂无数据' else ''
    capacity_kwh =soup.select('.dianchi .t1')[0].text if soup.select('.dianchi .t1')[0].text != '暂无数据' else ''
    max_power_kw =soup.select('.gonglv .t1')[0].text if soup.select('.gonglv .t1')[0].text != '暂无数据' else ''
    max_torque_nm =soup.select('.niuju .t1')[0].text if soup.select('.niuju .t1')[0].text != '暂无数据' else ''
    try:
        monthly_sales =soup.select('.monthsales .num')[0].text if len(soup.select('.monthsales .num')) > 0 else ''
    except AttributeError:
        monthly_sales = ''
        #yearly_sales =soup.select('.yearsales .num')[0].text if len(soup.select('.yearsales .num')) > 0 else ''
    try:
        monthly_price =soup.select('.monthprice .num')[0].text if soup.select('.monthprice .num')[0].text != '暂无数据'else ''
    except IndexError:
        monthly_price ='' 
    yearly_sales =soup.select('.yearsales .num')[0].text if len(soup.select('.yearsales .num')) > 0 else ''
    rating = soup.select('.star em')[0].text if soup.select('.star em')[0].text != '暂无' else ''
    review_count = soup.select('.txtnum')[0].text if soup.select('.txtnum')[0].text != '0篇口碑' else ''
    try:
        guide_price = soup.select('li:nth-child(1) .strong')[0].text.replace('\r', '').replace('\n', '').replace('\t','')
    except IndexError:
        guide_price =''
    return pd.DataFrame({'brand':[brand],'model':[model],'endurance_km':[endurance_km],'slow_charge_hour':[slow_charge_hour],
                         'fast_charge_hour':[fast_charge_hour],'capacity_kwh':[capacity_kwh],'max_power_kw':[max_power_kw],
                         'max_torque_nm':[max_torque_nm],'monthly_sales':[monthly_sales],'yearly_sales':[yearly_sales],
                         'monthly_price':[monthly_price],'rating':[rating],'review_count':[review_count],'guide_price':[guide_price]})

测试该函数。

In [10]:
model_url ="http://db.auto.sina.com.cn/3045/"
model_profile =crawl_model_profiles_by_model_url(model_url)
model_profile

,brand,model,endurance_km,slow_charge_hour,fast_charge_hour,capacity_kwh,max_power_kw,max_torque_nm,monthly_sales,yearly_sales,monthly_price,rating,review_count,guide_price
0,中华,中华H230 EV,150,3.5,1,24,,,,,,2.5分,1篇口碑,


In [11]:
model_urls = pd.read_csv("D:/data/e-auto/model_urls.csv", names = ["city_url"])
for model_url in model_urls['city_url']:
    model_profile = crawl_model_profiles_by_model_url(model_url)
    model_profile.to_csv('D:/data/e-auto/model_profiles.csv', mode = 'a', header = False, index = False)

In [12]:
model_urls = pd.read_csv("D:/data/e-auto/model_urls.csv", names = ["city_url"])
model_urls.head()

,city_url
0,http://db.auto.sina.com.cn/3460/
1,http://db.auto.sina.com.cn/3002/
2,http://db.auto.sina.com.cn/3854/
3,http://db.auto.sina.com.cn/4117/
4,http://db.auto.sina.com.cn/3694/


In [13]:
model_profiles = pd.read_csv("D:/data/e-auto/model_profiles.csv", names = ["model_url", "brand", "model", "endurance_km", "slow_charge_hour", 
        "fast_charge_hour", "capacity_kwh", "max_power_kw", "max_torque_nm", 
        "monthly_sales", "yearly_sales", "monthly_price", "rating", "review_count"])
model_profiles.head()

,model_url,brand,model,endurance_km,slow_charge_hour,fast_charge_hour,capacity_kwh,max_power_kw,max_torque_nm,monthly_sales,yearly_sales,monthly_price,rating,review_count
0,奥迪,奥迪Q2L e-tron,265.0,17.0,0.6,39.7,NaN,NaN,NaN,NaN,NaN,3.0分,NaN,22.68-23.73万
1,奥迪,奥迪A6L混动,54.0,NaN,2.5,14.1,270.0,500.0,NaN,NaN,NaN,4.0分,1篇口碑,50.8万
2,奥迪,奥迪A7混动,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0分,NaN,NaN
3,奥迪,A9 E-tron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0分,NaN,NaN
4,奥迪,AI:TRAIL quattro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0分,NaN,NaN
